In [35]:
import csv
from collections import Counter, defaultdict

In [3]:
with open('../data/SPARK.pruned.fam', 'r') as f, open('../data/spark.ped', 'w+') as outf:
    for line in f:
        fam_id, child_id, dad_id, mom_id, sex, affected = line.strip().split(' ')
        child_id = '%s_%s' % (fam_id, child_id)
        if dad_id != '0':
            dad_id = '%s_%s' % (fam_id, dad_id)
        if mom_id != '0':
            mom_id = '%s_%s' % (fam_id, mom_id)
        outf.write('\t'.join([fam_id, child_id, dad_id, mom_id, sex, affected]) + '\n')
        
        
        
        

In [38]:
# make peds per race

def pull_race(filename):
    sample_to_race = dict()
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        race_indices = [(header.index('race_asian'), 'asian'),
                        (header.index('race_african_amer'), 'african_amer'),
                        (header.index('race_native_amer'), 'native_amer'),
                        (header.index('race_native_hawaiian'), 'native_hawaiian'),
                        (header.index('race_white'), 'white'),
                        (header.index('race_other'), 'other'),
                        (header.index('race_more_than_one_calc'), 'more_than_one')]
        for pieces in reader:
            race = 'unknown'
            for index, r in race_indices:
                if pieces[index] == '1':
                    race = r
                sample_to_race['%s_%s' % (pieces[3], pieces[0])] = race
    return sample_to_race

sample_to_race = pull_race('../data/bghx_adult.csv')
sample_to_race.update(pull_race('../data/bghx_child.csv'))

In [39]:
print(Counter(sample_to_race.values()))

Counter({'white': 19652, 'more_than_one': 2603, 'african_amer': 1233, 'other': 893, 'asian': 490, 'native_amer': 127, 'unknown': 114, 'native_hawaiian': 24})


In [40]:
family_to_race = dict()
with open('../data/spark.ped', 'r') as f:
    for line in f:
        pieces = line.strip().split('\t')
        fam_id, samp_id = pieces[0], pieces[1]
        if samp_id in sample_to_race:
            if fam_id in family_to_race:
                if family_to_race[fam_id] != sample_to_race[samp_id]:
                    family_to_race[fam_id] = 'more_than_one'
            else:
                family_to_race[fam_id] = sample_to_race[samp_id]

In [42]:
for race in set(family_to_race.values()):
    with open('../data/spark.ped', 'r') as f, open('../data/spark_%s.ped' % race, 'w+') as outf:
        for line in f:
            pieces = line.strip().split('\t')
            fam_id = pieces[0]
            if (fam_id in family_to_race and family_to_race[fam_id] == race) or (race == 'unknown' and fam_id not in family_to_race):
                outf.write(line)
                
                
                